In [1]:
from keras.losses import logcosh
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
import tensorflow as tf


Using TensorFlow backend.


In [2]:
import time
import numpy as np
import pandas as pd
import copy
import random
from collections import defaultdict

np.random.seed(1)
np.set_printoptions(suppress=True)

In [3]:
class board():
    score =0
    lose = 0
    tile = 2
    def __init__(self):
        self.grid = np.array([np.zeros(4) for i in range(4)])
        i = np.random.randint(4)
        j = np.random.randint(4)
        self.grid[i,j] = 2
        k,l = i,j

        while(i==k and l==j):
            k = np.random.randint(4)
            l = np.random.randint(4)
        self.grid[k,l] = 2

### Shift, merge, and move constitute one move ###        
    def shift(self):
        current = np.array([np.zeros(4) for i in range(4)])
        for i in range(4):
            position = 0
            for j in range(4):
                if self.grid[i][j] != 0:
                    current[i][position] = self.grid[i][j]
                    position+=1
        self.grid = current
    def merge(self):
        #shift left, merge values, shift left
        self.shift()
        for i in range(4):
            for j in range(3):
                if self.grid[i][j] == self.grid[i][j+1] and self.grid[i][j] != 0:
                    self.grid[i][j] *= 2
                    self.score+= self.grid[i][j]
                    if (self.grid[i][j]) > self.tile:
                        self.tile = self.grid[i][j]
                    self.grid[i][j+1] = 0
        self.shift()
        
    def move(self, direction):
        #Merge everything left.  Use flip and transpose for other orientations
        self.temp = copy.deepcopy(self.grid)
        if direction == 'u':
            self.grid = np.transpose(self.grid)
            self.merge()
            self.grid = np.transpose(self.grid)
        elif direction == 'd':
            self.grid = np.flip(np.transpose(self.grid),1)
            self.merge()
            self.grid = np.transpose(np.flip(self.grid,1))
        elif direction == 'r':
            self.grid = np.flip(self.grid)
            self.merge()
            self.grid = np.flip(self.grid)
        else:
            self.merge()
        
### Check if the player has lost ###
    def lost(self):
        self.lose = 1
        for i in range(4):
            for j in range(4):
                if self.grid[i][j] == 0:
                    self.lose = 0 #Haven't lost yet
        for i in range(4):
            for j in range(4):
                try:
                    if (self.grid[i][j] == self.grid[i+1][j]):
                        self.lose = 0
                except:
                    pass
                try:
                    if (self.grid[i][j] == self.grid[i][j+1]):
                        self.lose = 0
                except:
                    pass
        
    def add_tile(self):
        possibilities = []
        for i in range(4):
            for j in range(4):
                if self.grid[i][j] == 0:
                    possibilities.append((i,j))
        value = np.random.choice([2,4], p=(.9,.1))
        if possibilities:
            a=possibilities[np.random.choice(len(possibilities))]
            self.grid[a] = value
    def legal_move(self):
        score = self.score
        legal_moves = []
        temp_grid = self.grid
        self.move('l')
        if (not np.array_equal(self.grid,temp_grid)):
            legal_moves.append('l')
        self.grid = temp_grid
        self.move('r')
        if (not np.array_equal(self.grid, temp_grid)):
            legal_moves.append('r')
        self.grid = temp_grid
        self.move('u')
        if (not np.array_equal(self.grid, temp_grid)):
            legal_moves.append('u')
        self.grid = temp_grid
        self.move('d')
        if (not np.array_equal(self.grid, temp_grid)):
            legal_moves.append('d')
        self.grid = temp_grid
        self.score = score
        return legal_moves
            
    def random_game(self):
        states = []
        while (not self.lose):
            move = np.random.choice(['l','r','u','d'])
            states.append((self.grid, move))

            self.move(move)
            self.lost()
            if (not np.array_equal(self.grid, self.temp)):
                self.add_tile()
#        print(self.grid)
#        print(self.score)

        return states
    def random_game_initial_score(self, depth, initial_move =0):
        #first iteration out of loop to get first move
        counter = 0
        if initial_move == 0:
            move = np.random.choice(self.legal_move())
            first_move = move
        else:
            move = initial_move
            first_move = initial_move
        self.move(move)
        self.lost()
        if (not np.array_equal(self.grid, self.temp)):
            self.add_tile()
        self.lost()
        while (not self.lose and counter < depth):
            counter += 1
#             if (not self.legal_move()):
#                 print(self.lose)
#                 print(self.grid)
#                 self.lost()
#                 print(self.lose)
            move = np.random.choice(self.legal_move())
            self.move(move)
            self.lost()
            if (not np.array_equal(self.grid, self.temp)):
                self.add_tile()
            self.lost()
        return first_move, self.score
   

In [4]:
def better_monte_carlo_new_random(game, depth):
    def getmove(game):
        grids = []
        possible_moves = game.legal_move()
#        print(possible_moves)
        for i in range(100):
            grids.append(copy.deepcopy(game))
        move_scores = defaultdict(list)
        i=0
        for games in grids:
            initial_move, score = games.random_game_initial_score(depth, possible_moves[i])
            move_scores[initial_move].append( score)
            i+=1
            i=i%(len(possible_moves))
        score = 0
        for key, value in move_scores.items():
            print(key,sum(value)/len(value))
            if (sum(value)/len(value) >score ):
                    move = key
                    score = sum(value)/len(value)
        print(move, score)
        return move
            

    states = []
    while (not game.lose):
        move = getmove(game)
        print(game.grid)
        #states.append((game.grid, move))
        game.move(move)
        game.lost()
        if (not np.array_equal(game.temp, game.grid)):
            game.add_tile()
        game.lost()
#    print(game.grid)
    print(game.score)
    print(game.tile)
    #return states

In [5]:
class DQN():
    def __init__(self, game):
        self.gamma = 0.99
        self.short_memory = np.array([])
        self.learning_rate = 0.005
        self.game = game
        self.epsilon = 0
        self.memory = []
        self.model = self.buildmodel()
    def buildmodel(self, weights=None):
        model = Sequential()
        model.add(Dense(16, activation='relu', input_dim = 16))
        model.add(Dropout(0.15))
#        model.add(Dense(output_dim=120, activation='relu'))
#         model.add(Dropout(0.15))
#         model.add(Dense(output_dim=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(4, activation='linear'))
        opt = Adam(lr=self.learning_rate)
        loss = tf.losses.huber_loss
        model.compile(loss=loss, optimizer=opt)
        

        print("Successfully built model")
        return model

        if weights:
            model.load_weights(weights)
        return model
    def get_state(self):
        return self.game.grid.reshape(1,16)
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def replay_new(self, memory):
        if len(memory) > 1000:
            minibatch = random.sample(memory, 1000)
        else:
            minibatch = memory
        for state, action, reward, next_state, lost in minibatch:
            pred_move = self.model.predict(state)
            pred_next_move = self.model.predict(next_state)
            target = reward

            if not lost:
                target = reward + self.gamma * np.max(pred_next_move[0])
            pred_move[0][action] = target
            
            self.model.fit(state, pred_move, epochs=10, verbose=0)
    def train_short_memory(self, state, action, reward, next_state, lost, games_counter):
        pred_move = self.model.predict(state)
        pred_next_move = self.model.predict(next_state)
        target = reward

        if not lost:
            target = reward + self.gamma * np.max(pred_next_move[0])
        pred_move[0][action] = target
            
        self.model.fit(state, pred_move, epochs=10, verbose=0)
        

In [6]:
def int_to_move(i):
    if i == 0:
        return 'l'
    if i ==1:
        return 'r'
    if i == 2:
        return 'u'
    if i == 3:
        return 'd'
def move_to_int(i):
    if i=='l':
        return 0
    if i == 'r':
        return 1
    if i == 'u':
        return 2
    if i == 'd':
        return 3
def run(dqn):
    games_counter = 0
    score_plot = []
    counter_plot = []
    record = 0
    while games_counter < 3000:
        dqn.game = board()
        while not dqn.game.lose:
            dqn.epsilon = 188*(.97**games_counter)
            
            state_old = dqn.get_state()
            prev_score = dqn.game.score
            if np.random.randint(0,200)< dqn.epsilon:
                legal_moves = dqn.game.legal_move()
                final_move = np.random.choice(legal_moves)
            else:
                prediction = dqn.model.predict(state_old.reshape(1,16))
                final_move = np.argmax(prediction[0])
            final_move = int_to_move(final_move)
            temp = copy.deepcopy(dqn.game.grid)

#            print("+++++++++++++++++++++++++++++++")
#            print(dqn.game.grid, final_move)
            dqn.game.move(final_move)
            if (not np.array_equal(temp, dqn.game.grid)):
                game.add_tile()
            dqn.game.add_tile()
            final_move = move_to_int(final_move)
            state_new = dqn.get_state()
            if dqn.game.score == prev_score:
                reward = -1
            else:
                reward = np.log2(dqn.game.score - prev_score)
            done = dqn.game.lost()
            
            if done:
                reward -= 1
            dqn.train_short_memory(state_old, final_move, reward, state_new, done, games_counter)
            
            dqn.remember(state_old, final_move, reward, state_new, done)
            
        dqn.replay_new(dqn.memory)
        games_counter +=1
        print("game", games_counter, "score", dqn.game.score)
    dqn.model.save_weights('weights.hdf5')
            
            
        